In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup as bs

import requests
import json
from random import randint
import os
import time
import json
import pickle

In [2]:
%cd ..
%cd demo Mativ
all_anime_df = pd.read_csv(os.path.join("data", "tags_anime.csv"))
%cd ..
%cd demo Victor
all_anime_df


path = os.path.join("data", "detalles_anime_1.pickle")
with open(path, "rb") as archivo:
    detalles_anime = pickle.load(archivo)

e:\Github\datos\Proyecto-Grupo-12
e:\Github\datos\Proyecto-Grupo-12\demo Mativ
e:\Github\datos\Proyecto-Grupo-12
e:\Github\datos\Proyecto-Grupo-12\demo Victor


# Match con MyAnimeList

In [ ]:
all_anime_df["Anime"] = all_anime_df["Anime"].str.lower()
all_anime_df["Anime"] = all_anime_df["Anime"].str.replace(r"[^a-zA-Z0-9]", "", regex=True)

In [79]:
import re

pd.options.mode.chained_assignment = None  # default='warn'
titulos = pd.DataFrame()
not_found = {}
for anime_id, value in detalles_anime.items():
    details = value["detalles"]
    lista_titulos = [details["title"]] + (details["alternative_titles"]["synonyms"]) + [(details["alternative_titles"]["en"])]
    for titulo in lista_titulos:
        mini = all_anime_df[all_anime_df["Anime"] == re.sub("[^a-zA-Z0-9]", "", titulo.lower())]
        if not mini.empty:
            break
    else: # No se encontro el anime
        not_found[anime_id] = lista_titulos
        continue
    mini["MAL_ID"] = anime_id
    titulos = pd.concat([titulos, mini])
titulos

,Anime,URL,MAL_ID
112,cowboybebop,https://www.anime-planet.com/anime/cowboy-bebop,1
13925,puzzledragonsx,https://www.anime-planet.com/anime/puzzle-and-...,32772
353,cowboybebopthemovie,https://www.anime-planet.com/anime/cowboy-bebo...,5
422,trigun,https://www.anime-planet.com/anime/trigun,6
4663,witchhunterrobin,https://www.anime-planet.com/anime/witch-hunte...,7
...,...,...,...
11401,tidelineblue,https://www.anime-planet.com/anime/tide-line-blue,473
2204,macrossplus,https://www.anime-planet.com/anime/macross-plus,474
2839,honobonolog,https://www.anime-planet.com/anime/honobono-log,33241
3832,ginbankaleidoscope,https://www.anime-planet.com/anime/ginban-kale...,476


In [66]:
all_anime_df[all_anime_df["URL"] == "https://www.anime-planet.com/anime/aikatsu-nerawareta-mahou-no-aikatsu-card"]

,Anime,URL
4161,aikatsu! nerawareta mahou no aikatsu card,https://www.anime-planet.com/anime/aikatsu-ner...


In [80]:
not_found

{49: ['Aa! Megami-sama!', 'Ah! My Goddess (OVA)', 'Oh! My Goddess'],
 52: ['Kidou Tenshi Angelic Layer', 'Battle Doll Angelic Layer'],
 62: ['D.C.: Da Capo', 'D.C.~Da Capo~'],
 66: ['Azumanga Daiou The Animation',
  'Azumanga Daiou',
  'Azumanga Daioh: The Animation'],
 32867: ['Bungou Stray Dogs 2nd Season', 'Bungo Stray Dogs 2'],
 100: ['Shin Shirayuki-hime Densetsu Prétear',
  'Shin Shirayuki-hime Densetsu Pretear',
  'Prétear: The New Legend of Snow White'],
 112: ['Chou Henshin Cosprayers',
  'Super Transformation Cosprayers',
  'The Cosmopolitan Prayers'],
 118: ['Shinpi no Sekai El-Hazard Season 2',
  'El Hazard 2: The Magnificent World'],
 123: ['Fushigi Yuugi', 'Fushigi Yugi', 'Curious Play', 'Mysterious Play'],
 124: ['Fushigi Yuugi: Eikouden',
  'Fushigi Yuugi OVA 3',
  'Mysterious Play: Eikoden'],
 32900: ['Mahouka Koukou no Rettousei Movie: Hoshi wo Yobu Shoujo',
  'Gekijouban Mahouka Koukou no Rettousei',
  'The Irregular at Magic High School The Movie - The Girl Who Summ

In [12]:
detalles.items()

dict_items([(110, {'detalles': {'id': 110, 'title': 'Chuuka Ichiban!', 'main_picture': {'medium': 'https://cdn.myanimelist.net/images/anime/6/75283.jpg', 'large': 'https://cdn.myanimelist.net/images/anime/6/75283l.jpg'}, 'alternative_titles': {'synonyms': ['Cooking Master Boy'], 'en': '', 'ja': '中華一番'}}}), (128, {'detalles': {'id': 128, 'title': 'Gate Keepers 21', 'main_picture': {'medium': 'https://cdn.myanimelist.net/images/anime/6/75569.jpg', 'large': 'https://cdn.myanimelist.net/images/anime/6/75569l.jpg'}, 'alternative_titles': {'synonyms': ['Gatekeepers 21'], 'en': '', 'ja': 'ゲートキーパーズ21'}}}), (134, {'detalles': {'id': 134, 'title': 'Gunslinger Girl', 'main_picture': {'medium': 'https://cdn.myanimelist.net/images/anime/5/3088.jpg', 'large': 'https://cdn.myanimelist.net/images/anime/5/3088l.jpg'}, 'alternative_titles': {'synonyms': ['Gunslinger Girls'], 'en': 'Gunslinger Girl', 'ja': 'ガンスリンガー・ガール'}}}), (159, {'detalles': {'id': 159, 'title': 'Boukyaku no Senritsu', 'main_picture': 

In [26]:
anime_id = 33254
value = detalles[anime_id]

anime_details = value["detalles"]
lista_titulos = [anime_details["title"]] + (anime_details["alternative_titles"]["synonyms"]) + list(anime_details["alternative_titles"]["en"])

# We need to find wether any of the titles in lista_titulos is in the all_anime_df["Anime"] column
# If it is, we need to get the index of the row where it is

for titulo in lista_titulos:
    mini = all_anime_df[all_anime_df["Anime"] == titulo]
    if not mini.empty:
        break
mini

,Anime,URL
1196,The Heroic Legend of Arslan,https://www.anime-planet.com/anime/the-heroic-...


In [68]:
re.sub("[^a-zA-Z0-9 ]", "", "AabB123 - %$#$")

'AabB123  '

In [81]:
detalles_anime[32915]

{'detalles': {'id': 32915,
  'title': 'Durarara!!x2 Ketsu: Dufufufu!!',
  'main_picture': {'medium': 'https://cdn.myanimelist.net/images/anime/2/78979.jpg',
   'large': 'https://cdn.myanimelist.net/images/anime/2/78979l.jpg'},
  'alternative_titles': {'synonyms': ['Durarara!!x2 Ketsu OVA',
    'Durarara!!x2 Ketsu Special',
    'Durarara!!x2 Ketsu Episode 19.5'],
   'en': 'Durarara!! x2 Ketsu: Dufufufu!!',
   'ja': 'デュラララ!!×２ 結\u3000第19.5話「デュフフフ!!」'},
  'start_date': '2016-07-27',
  'end_date': '2016-07-27',
  'synopsis': "With his handsome slim-fit suit, characteristic glossy shades, and a casual cigarette in mouth, word is that the remarkably strong Shizuo Heiwajima rules Ikebukoro's streets. When ordinary bartender Shizuo Nobusuma hears these rumors, he realizes he could make his own life a bit easier: as a con artist. \n\nDonning a blond wig and venturing into Ikebukuro assuming Shizuo's identity, the impostor is confident he is fooling everyone into doing his bidding. But as the di

# Tags

In [ ]:
def extraer_animes(animes: pd.Dataframe, head):
    # Tiempo de espera aleatorio entre 1 y 3 segundos.
    extension = animes.copy()
    extension["Tags"] = ""
    extension["Alts"] = ""

    for index, row in animes.iterrows():
        # Tiempo de espera aleatorio entre 1 y 3 segundos.
        time.sleep(randint(1, 3))
        anime_url = row["URL"]
        anime_title = row["Anime"]

        while True:
            try:
                anime_response = requests.get(url=anime_url, headers=head)

                # Manejar Status codes.
                if anime_response.status_code != 200: # Error desconocido
                    print(anime_response.status_code)
                    print(anime_response.json())
                    print(anime_title, anime_url)
                    time.sleep(60)
                    continue

                anime_soup = bs(anime_response.content, "html.parser")
                
                # Tags del anime
                meta_tags = anime_soup.find_all("meta", {"property": "video:tag"})
                tags = [tag["content"] for tag in meta_tags]
                
                # Titulos alternativos
                alt_titles = anime_soup.find_all("h2", {"class": "aka"})[0].contents[0]

                extension.at[index, "Tags"] = tags
                extension.at[index, "Alts"] = alt_titles
                break

            # Manejar errores
            except KeyboardInterrupt: # Usuario saltado por algun error etc.
                print("anime saltado")
                print(anime_response.status_code)
                print(anime_response.json())
                print(anime_title, anime_url)
                break
            except ConnectionError:
                print("Error de internet")
                print(anime_title, anime_url)
                time.sleep(300)
                continue

    return extension

In [13]:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.81 Safari/537.36 Edg/94.0.992.50",
    "Accept-Language": "en-US,en;q=0.5",
    "Accept-Encoding": "gzip, deflate",
    "Connection": "keep-alive",
    "Upgrade-Insecure-Requests": "1",
    "TE": "Trailers",
}
anime_response = requests.get(anime_url, headers=headers)

In [14]:
anime_response

<Response [200]>

In [15]:
anime_soup = bs(anime_response.content, "html.parser")

In [24]:
[tag["content"] for tag in anime_soup.find_all("meta", {"property": "video:tag"})]

['Action',
 'Drama',
 'Fantasy',
 'Shounen',
 'Dark Fantasy',
 'Military',
 'War',
 'Explicit Violence',
 'Mature Themes',
 'Physical Abuse',
 'Suicide',
 'Based on a Manga']

In [34]:
# Find h2 class ="aka"
aka = anime_soup.find_all("h2", {"class": "aka"})[0]
aka

<h2 class="aka">
Alt titles: Attack on Titan The Final Season: Part III, Shingeki no Kyojin The Final Season: Kanketsu-hen
</h2>

In [35]:
# get content of aka
aka_content = aka.contents[0]

In [36]:
aka_content

'\nAlt titles: Attack on Titan The Final Season: Part III, Shingeki no Kyojin The Final Season: Kanketsu-hen\n'